In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from utils.ModelingUtils import make_ml_target, MODEL_FEATURES
from utils.PreprocessingUtils import get_route_names, fix_polish_chars
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFECV
import joblib

In [2]:
data = pd.read_parquet('../data/prepared_data_with_weather.parquet')
data = make_ml_target(data)
# data = get_route_names(data)
# routes = data['sorted_stations'].unique()

In [8]:
# n = 1
# for route in routes:
#     print(n)
#     subset = data[data['sorted_stations']==route].copy()

#     y = subset['ML_TARGET'].values
#     X = subset[MODEL_FEATURES]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
#     numeric_features = X_train.select_dtypes(include=['number']).columns
#     categorical_features = X_train.select_dtypes(exclude=['number']).columns

#     numeric_transformer = Pipeline(steps=[('imputer', KNNImputer()),])
#     categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numeric_transformer, numeric_features),
#             ('cat', categorical_transformer, categorical_features)
#         ])

#     pipe = Pipeline([
#         ('preprocessor', preprocessor),
#     ])

#     pipe.fit(X_train)
#     joblib.dump(pipe, f'files/pipe_route_nr{n}.pkl')

#     X_train_processed = pipe.transform(X_train)
#     X_test_processed = pipe.transform(X_test)

#     # Get column names after transformation
#     transformed_numeric_columns = preprocessor.named_transformers_['num'].named_steps['imputer'].get_feature_names_out(input_features=numeric_features)
#     transformed_categorical_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(input_features=categorical_features)

#     # Combine numeric and categorical columns
#     transformed_columns = list(transformed_numeric_columns) + list(transformed_categorical_columns)

#     # Create DataFrame with processed data and column names
#     processed_df_train = pd.DataFrame(X_train_processed, columns=transformed_columns)
#     processed_df_test = pd.DataFrame(X_test_processed, columns=transformed_columns)

#     processed_df_train = pd.concat([processed_df_train, pd.Series(y_train).rename('ML_TARGET')], axis=1)
#     processed_df_test = pd.concat([processed_df_test, pd.Series(y_test).rename('ML_TARGET')], axis=1)

#     processed_df_train.to_parquet(f'../data/preprocessed_for_modeling/train_data_route_nr{n}.parquet')
#     processed_df_test.to_parquet(f'../data/preprocessed_for_modeling/test_data_route_nr{n}.parquet')
    

    
#     n+=1
    

In [3]:
y = data['ML_TARGET'].values
X = data[MODEL_FEATURES]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

numeric_features = X_train.select_dtypes(include=['number']).columns
categorical_features = X_train.select_dtypes(exclude=['number']).columns

In [4]:
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer()),
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipe = Pipeline([
    ('preprocessor', preprocessor),
])

In [5]:
pipe.fit(X_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer())]),
                                                  Index(['station_count_on_curr_station', 'full_route_station_count',
       'distance_to_near_city_station', 'distance_to_prev_station',
       'distance_from_start', 'distance_to_final', 'full_route_distance',
       'days_until_christmas', 'weekday_Friday', 'weekda...
       'stop_duration_lag4', 'stop_duration_lag5', 'stop_duration_lag6',
       'temp', 'feelslike', 'humidity', 'dew', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'pressure',
       'visibility', 'cloudcover', 'solarradiation', 'solarenergy', 'uvindex'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['near_city_station_name', 'preciptype'], dtype='object'))]))])

In [6]:
joblib.dump(pipe, 'files/fitted_pipeline.pkl')

['files/fitted_pipeline.pkl']

In [7]:
X_train_processed = pipe.transform(X_train)
X_test_processed = pipe.transform(X_test)

KeyboardInterrupt: 

### processed data to df

In [ ]:
# Get column names after transformation
transformed_numeric_columns = preprocessor.named_transformers_['num'].named_steps['imputer'].get_feature_names_out(input_features=numeric_features)
transformed_categorical_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(input_features=categorical_features)

# Combine numeric and categorical columns
transformed_columns = list(transformed_numeric_columns) + list(transformed_categorical_columns)

# Create DataFrame with processed data and column names
processed_df_train = pd.DataFrame(X_train_processed, columns=transformed_columns)
processed_df_test = pd.DataFrame(X_test_processed, columns=transformed_columns)

In [ ]:
processed_df_train = pd.concat([processed_df_train, pd.Series(y_train).rename('ML_TARGET')], axis=1)
processed_df_test = pd.concat([processed_df_test, pd.Series(y_test).rename('ML_TARGET')], axis=1)

In [ ]:
processed_df_train.to_parquet('../data/preprocessed_for_modeling/train_data.parquet')
processed_df_test.to_parquet('../data/preprocessed_for_modeling/test_data.parquet')

array([0, 1])

In [25]:
# estimator = RandomForestClassifier(random_state=123)
# rfecv = RFECV(estimator=estimator, step=1, cv=5, scoring='accuracy')
# rfecv.fit(X_train_processed, y_train)

In [29]:
# processed_df_train.drop('ML_TARGET',axis=1,inplace=True)

In [38]:
# model = CatBoostClassifier(max_depth=10, random_seed=123)
# model.fit(processed_df_train, y_train)

In [38]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.99      0.93    193326
           1       0.57      0.10      0.17     26277
           2       0.54      0.11      0.19      6840
           3       0.68      0.05      0.10      1199

    accuracy                           0.86    227642
   macro avg       0.66      0.32      0.35    227642
weighted avg       0.82      0.86      0.81    227642


In [45]:
roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

0.8807862587345204

In [42]:
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(model.classes_)

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curve for each class
plt.figure(figsize=(8, 6))
lw = 2
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

# Plot micro-average ROC curve
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multiclass Classification')
plt.legend(loc="lower right")
plt.show()

ValueError: multiclass format is not supported

In [41]:
y_test

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
model = RandomForestClassifier(random_state=123)

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

cv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
cv.fit(X_train, y_train)